# HandsOn Week 13
Welcome to handsOn week 13: graph in distributed system, using GraphFrames. To use GraphFrames, you need to run ```pyspark --packages graphframes:graphframes:0.8.0-spark2.4-s_2.11``` (adjust with your Spark version. That script uses Spark 2.4.x, the one used in our VM, and it will download the graphframes package automatically, thus, make sure you have internet connection when launching the script).

Note: you can use any Spark & GraphFrames API (without building from-the-scratch).

### Read The Dataset and Build The Graph

In [2]:
from pyspark.sql.types import *
from graphframes import *

def create_transport_graph(nodes_filePath, edges_filePath):
    node_fields = [
        StructField("id", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    nodes = spark.read.csv(nodes_filePath, header=True,schema=StructType(node_fields))
    edge_fields = [
        StructField("src", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("cost", FloatType(), True)
    ]
    rels = spark.read.csv(edges_filePath, header=True, schema=StructType(edge_fields))
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst")
                     .select("src", "dst", "relationship", "cost"))
    relationships = rels.union(reversed_rels)
    return GraphFrame(nodes, relationships)

g = create_transport_graph("./transport-nodes.csv", "./transport-relationships.csv")
g.vertices.show()
g.edges.show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam| 52.37919| 4.899431|    821752|
|         Utrecht|52.092876|  5.10448|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+

+----------------+----------------+------------+-----+
|             src|             dst|relationship| cost|
+----------------+----------------+------------+-----+
|       Amsterdam|         Utrecht

## Milestone 01
1. Find source-destination airport pair/pairs with maximum cost
2. Find source-destination airport pair/pairs with minimum cost
3. Calculate the average cost

In [3]:
# write your code here
# iterate through the edge and find the minimum, maximum, and average cost

all_edge_cost = g.edges.select("cost").collect()
minimum = g.edges.select("cost").collect()[0][0]
maximum = g.edges.select("cost").collect()[0][0]
total = 0
for edge in all_edge_cost:
    if(edge[0] < minimum):
        minimum = edge[0]
    
    if(edge[0] > maximum):
        maximum = edge[0]
    total += edge[0]


average = total / len(all_edge_cost)
# print the result
print("Minimum cost: {}".format(minimum))
print("Maximum cost: {}".format(maximum))
print("Average cost: {}".format(average))


Minimum cost: 22.0
Maximum cost: 369.0
Average cost: 91.33333333333333


### Milestone 02
1. Find flight routes that have no direct connection

In [4]:
# write your code here

all_cities = g.vertices.select("id").collect()
all_connections = g.edges.select("src", "dst").collect()

indirect_connections = []
# iterate through the cities twice.  
for city1 in all_cities:
    for city2 in all_cities:
        if(city1[0] != city2[0]):
            if((city1[0], city2[0]) not in all_connections):
                if((city2[0], city1[0]) not in all_connections):
                    indirect_connections.append((city1[0], city2[0]))

print("Number of indirect connections: {}".format(len(indirect_connections)))
print("Indirect connections: {}".format(indirect_connections))


Number of indirect connections: 102
Indirect connections: [('Amsterdam', 'Doncaster'), ('Amsterdam', 'Hoek van Holland'), ('Amsterdam', 'Felixstowe'), ('Amsterdam', 'Ipswich'), ('Amsterdam', 'Colchester'), ('Amsterdam', 'London'), ('Amsterdam', 'Rotterdam'), ('Amsterdam', 'Gouda'), ('Utrecht', 'Den Haag'), ('Utrecht', 'Immingham'), ('Utrecht', 'Doncaster'), ('Utrecht', 'Hoek van Holland'), ('Utrecht', 'Felixstowe'), ('Utrecht', 'Ipswich'), ('Utrecht', 'Colchester'), ('Utrecht', 'London'), ('Utrecht', 'Rotterdam'), ('Den Haag', 'Utrecht'), ('Den Haag', 'Immingham'), ('Den Haag', 'Doncaster'), ('Den Haag', 'Felixstowe'), ('Den Haag', 'Ipswich'), ('Den Haag', 'Colchester'), ('Den Haag', 'London'), ('Immingham', 'Utrecht'), ('Immingham', 'Den Haag'), ('Immingham', 'Hoek van Holland'), ('Immingham', 'Felixstowe'), ('Immingham', 'Ipswich'), ('Immingham', 'Colchester'), ('Immingham', 'London'), ('Immingham', 'Rotterdam'), ('Immingham', 'Gouda'), ('Doncaster', 'Amsterdam'), ('Doncaster', 'Utre

### Milestone 03
1. Find the most important airport (you can use any measurement to judge the importance level of airports, and please describe why you choose it -the measurement-)

In [5]:
# fint the city that show up the most in the relationship. if it has the most traffic, it probably is the most important city

# all_connections
city_count = {}
for connection in all_connections:
    if(connection[0] in city_count):
        city_count[connection[0]] += 1
    else:
        city_count[connection[0]] = 1
        
    if(connection[1] in city_count):
        city_count[connection[1]] += 1
    else:
        city_count[connection[1]] = 1

max_city = ""
max_count = 0
for city in city_count:
    if(city_count[city] > max_count):
        max_count = city_count[city]
        max_city = city

print("City that show up the most in the relationship: {}".format(max_city))
    

City that show up the most in the relationship: Den Haag


### Milestone 04
1. Find the sortest path based on "node" -path with fewest nodes- from Amsterdam to London 

In [6]:
# do dijkstra algorithm to find the shortest path from city Amsterdam to London


edge = []
for connection in all_connections:
    edge.append([connection[0], connection[1], 1])


# get all unique cities from edge
nodes = []
for connection in edge:
    if(connection[0] not in nodes):
        nodes.append(connection[0])
    if(connection[1] not in nodes):
        nodes.append(connection[1])
    


# all connection has a cost of 1
def dijkstra(edge, nodes, start, goal):
    # edge is an array of array start, end, cost
    # nodes is an array of all nodes

    visited = []
    unvisited = nodes
    distance = {}
    previous = {}
    for node in nodes:
        distance[node] = float("inf")
        previous[node] = None
    distance[start] = 0

    while unvisited:
        current = min(unvisited, key=distance.get)
        if current == goal:
            break
        unvisited.remove(current)
        visited.append(current)
        for eachedge in edge:
            src = eachedge[0]
            dst = eachedge[1]
            cost = eachedge[2]
            if src == current:
                if dst not in visited:
                    new_distance = distance[current] + cost
                    if new_distance < distance[dst]:
                        distance[dst] = new_distance
                        previous[dst] = current
                    
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = previous[current]
    path.append(start)
    path.reverse()
    return path






path = dijkstra(edge, nodes, "Amsterdam", "London")
print("path: ", path)

path:  ['Amsterdam', 'Immingham', 'Doncaster', 'London']


### Bonus
1. Find the sortest path based on "cost value" -see 'cost' column in the edge/relationship dataframe - from Amsterdam to London 

In [7]:
# do dijkstra algorithm to find the shortest path from city Amsterdam to London with cost


all_connections_with_cost = g.edges.select("src", "dst", "cost").collect()
edge = []

for connection in all_connections_with_cost:
    edge.append([connection[0], connection[1], connection[2]])


# get all unique cities from edge
nodes = []
for connection in edge:
    if(connection[0] not in nodes):
        nodes.append(connection[0])
    if(connection[1] not in nodes):
        nodes.append(connection[1])
    


# all connection has a cost of 1
def dijkstra(edge, nodes, start, goal):
    # edge is an array of array start, end, cost
    # nodes is an array of all nodes

    visited = []
    unvisited = nodes
    distance = {}
    previous = {}
    for node in nodes:
        distance[node] = float("inf")
        previous[node] = None
    distance[start] = 0

    while unvisited:
        current = min(unvisited, key=distance.get)
        if current == goal:
            break
        unvisited.remove(current)
        visited.append(current)
        for eachedge in edge:
            src = eachedge[0]
            dst = eachedge[1]
            cost = eachedge[2]
            if src == current:
                if dst not in visited:
                    new_distance = distance[current] + cost
                    if new_distance < distance[dst]:
                        distance[dst] = new_distance
                        previous[dst] = current
                    
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = previous[current]
    path.append(start)
    path.reverse()
    return path





path = dijkstra(edge, nodes, "Amsterdam", "London")
print("path: ", path)


path:  ['Amsterdam', 'Den Haag', 'Hoek van Holland', 'Felixstowe', 'Ipswich', 'Colchester', 'London']


# Submission
Submit this ```ipynb``` file to the course portal, with format: ```HandsOnWeek13_NIM_NamaLengkap.ipynb```. Make sure when submitting this file, each code cell has the outputs (not blank).

Christopher Jeffrey 13520055